## Telecom Churn: Logistic Regression with PCA

### Objective and Assumptions
-- This project is based on the Indian and Southeast Asian market.So will focus on Prepaid mode.

-- As per business we will build model based on Usage-based churn i.e. Customers who have not done any usage, either incoming or outgoing - in terms of calls, internet etc. over a period of time

--We will predict churn only for the high-value customers.

--We assume that there are three phases of customer lifecycle : Good (June- July), Action(Aug) and Churn (Sep)

With 226 predictor variables and 99999 observation, The business objective is to predict the churn in the last (i.e. the ninth) month using the data (features) from the first three months.


### Importing Data

In [128]:
# Importing Pandas and NumPy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [139]:
# Importing datasets
raw_data = pd.read_csv("telecom_churn_data.csv")

In [130]:
print(len(raw_data))

99999


In [131]:
# Let's see the head of our master dataset
raw_data.head()

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,arpu_6,...,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
0,7000842753,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,197.385,...,0,1.0,1.0,1.0,NaN,968,30.4,0.0,101.20,3.58
1,7001865778,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,34.047,...,0,NaN,1.0,1.0,NaN,1006,0.0,0.0,0.00,0.00
2,7001625959,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,167.690,...,0,NaN,NaN,NaN,1.0,1103,0.0,0.0,4.17,0.00
3,7001204172,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,221.338,...,0,NaN,NaN,NaN,NaN,2491,0.0,0.0,0.00,0.00
4,7000142493,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,261.636,...,0,0.0,NaN,NaN,NaN,1526,0.0,0.0,0.00,0.00


In [132]:
raw_data.describe()

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,arpu_9,onnet_mou_6,...,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
count,9.999900e+04,99999.0,98981.0,98981.0,98981.0,99999.000000,99999.000000,99999.000000,99999.000000,96062.000000,...,99999.000000,25153.000000,25571.000000,26339.000000,25922.000000,99999.000000,99999.000000,99999.000000,99999.000000,99999.000000
mean,7.001207e+09,109.0,0.0,0.0,0.0,282.987358,278.536648,279.154731,261.645069,132.395875,...,0.084581,0.914404,0.908764,0.890808,0.860968,1219.854749,68.170248,66.839062,60.021204,3.299373
std,6.956694e+05,0.0,0.0,0.0,0.0,328.439770,338.156291,344.474791,341.998630,297.207406,...,0.650457,0.279772,0.287950,0.311885,0.345987,954.733842,267.580450,271.201856,253.938223,32.408353
min,7.000000e+09,109.0,0.0,0.0,0.0,-2258.709000,-2014.045000,-945.808000,-1899.505000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,180.000000,0.000000,0.000000,0.000000,0.000000
25%,7.000606e+09,109.0,0.0,0.0,0.0,93.411500,86.980500,84.126000,62.685000,7.380000,...,0.000000,1.000000,1.000000,1.000000,1.000000,467.000000,0.000000,0.000000,0.000000,0.000000
50%,7.001205e+09,109.0,0.0,0.0,0.0,197.704000,191.640000,192.080000,176.849000,34.310000,...,0.000000,1.000000,1.000000,1.000000,1.000000,863.000000,0.000000,0.000000,0.000000,0.000000
75%,7.001812e+09,109.0,0.0,0.0,0.0,371.060000,365.344500,369.370500,353.466500,118.740000,...,0.000000,1.000000,1.000000,1.000000,1.000000,1807.500000,0.000000,0.000000,0.000000,0.000000
max,7.002411e+09,109.0,0.0,0.0,0.0,27731.088000,35145.834000,33543.624000,38805.617000,7376.710000,...,49.000000,1.000000,1.000000,1.000000,1.000000,4337.000000,12916.220000,9165.600000,11166.210000,2618.570000


In [133]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Columns: 226 entries, mobile_number to sep_vbc_3g
dtypes: float64(179), int64(35), object(12)
memory usage: 172.4+ MB


In [134]:
#raw_data.columns.tolist()

### Checking for Missing Values and Inputing Them

In [135]:
## Checking row with all null values
raw_data.isnull().all(axis = 0).sum()

0

In [9]:
#raw_data1 = raw_data[(raw_data['total_rech_amt_6'] != 0) & (raw_data['total_rech_amt_7'] != 0)]

#raw_data1 = raw_data

#raw_data1 = raw_data1[(raw_data1['total_rech_num_6'] != 0) & (raw_data1['total_rech_num_7'] != 0)]

In [10]:
raw_data['vol_3g_mb_7'].dtype

dtype('float64')

In [140]:
## Checking all churn parameter in 6th and 7th month, if 0 value of churn parameter in good month assuming customer is already churned
raw_data1 = raw_data[~((raw_data['total_og_mou_6'] == 0) & (raw_data['total_og_mou_7'] == 0) & (raw_data['total_ic_mou_6'] == 0) & (raw_data['total_ic_mou_7'] == 0) & (raw_data['vol_2g_mb_6'] == 0) & (raw_data['vol_2g_mb_7'] == 0) & (raw_data['vol_3g_mb_6'] == 0) & (raw_data['vol_3g_mb_7'] == 0))]

## Checking all churn parameter in 6th,7th and 8th month, if 0 value of churn parameter in good month assuming customer is already churned

#raw_data1 = raw_data[~((raw_data['total_og_mou_6'] == 0) & (raw_data['total_og_mou_7'] == 0) & (raw_data['total_og_mou_8'] == 0) & (raw_data['total_ic_mou_6'] == 0) & (raw_data['total_ic_mou_7'] == 0) & (raw_data['total_ic_mou_8'] == 0) & (raw_data['vol_2g_mb_6'] == 0) & (raw_data['vol_2g_mb_7'] == 0) & (raw_data['vol_2g_mb_8'] == 0) & (raw_data['vol_3g_mb_6'] == 0) & (raw_data['vol_3g_mb_7'] == 0) & (raw_data['vol_3g_mb_8'] == 0))]

## Checking all churn parameter in 6th,7th and 8th month, if 0 value of churn parameter in good month assuming customer is already churned

#raw_data1 = raw_data[~((raw_data['total_og_mou_6'] == 0) & (raw_data['total_og_mou_7'] == 0) & (raw_data['total_og_mou_8'] == 0) & (raw_data['total_og_mou_9'] == 0) & (raw_data['total_ic_mou_6'] == 0) & (raw_data['total_ic_mou_7'] == 0) & (raw_data['total_ic_mou_8'] == 0) & (raw_data['total_ic_mou_9'] == 0) & (raw_data['vol_2g_mb_6'] == 0) & (raw_data['vol_2g_mb_7'] == 0) & (raw_data['vol_2g_mb_8'] == 0) & (raw_data['vol_2g_mb_9'] == 0) & (raw_data['vol_3g_mb_6'] == 0) & (raw_data['vol_3g_mb_7'] == 0) & (raw_data['vol_3g_mb_8'] == 0) & (raw_data['vol_3g_mb_9'] == 0))]

In [244]:
print(len(raw_data1))

97153


In [142]:
print (100 *((len(raw_data) - (len(raw_data1)))/ len(raw_data)))

2.846028460284603


Around 3% data loss with above cleaning

In [143]:
#Checking unique value of mobile numbers to remove duplicate
unique_no = raw_data1.mobile_number.unique().tolist()
raw_data1= raw_data1[raw_data1['mobile_number'].isin (unique_no)]

In [144]:
#Checking column with more than 70% null data
col_null = raw_data1.columns[round(100*(raw_data1.isnull().sum()/len(raw_data1.index)), 2) > 70.00 ]
col_null

Index(['date_of_last_rech_data_6', 'date_of_last_rech_data_7',
       'date_of_last_rech_data_8', 'date_of_last_rech_data_9',
       'total_rech_data_6', 'total_rech_data_7', 'total_rech_data_8',
       'total_rech_data_9', 'max_rech_data_6', 'max_rech_data_7',
       'max_rech_data_8', 'max_rech_data_9', 'count_rech_2g_6',
       'count_rech_2g_7', 'count_rech_2g_8', 'count_rech_2g_9',
       'count_rech_3g_6', 'count_rech_3g_7', 'count_rech_3g_8',
       'count_rech_3g_9', 'av_rech_amt_data_6', 'av_rech_amt_data_7',
       'av_rech_amt_data_8', 'av_rech_amt_data_9', 'arpu_3g_6', 'arpu_3g_7',
       'arpu_3g_8', 'arpu_3g_9', 'arpu_2g_6', 'arpu_2g_7', 'arpu_2g_8',
       'arpu_2g_9', 'night_pck_user_6', 'night_pck_user_7', 'night_pck_user_8',
       'night_pck_user_9', 'fb_user_6', 'fb_user_7', 'fb_user_8', 'fb_user_9'],
      dtype='object')

In [145]:
#Dropping few col from above which will not make much impact on analysis, Keeping rest to analyze further
raw_data1.drop(['date_of_last_rech_data_6', 'date_of_last_rech_data_7',
       'date_of_last_rech_data_8', 'date_of_last_rech_data_9',
       'arpu_2g_9', 'night_pck_user_6', 'night_pck_user_7', 'night_pck_user_8',
       'night_pck_user_9', 'fb_user_6', 'fb_user_7', 'fb_user_8', 'fb_user_9'], axis = 1 , inplace = True)

In [152]:
raw_data1.head()

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,arpu_6,...,sep_vbc_3g,total_data_rech_amt_6,total_data_rech_amt_7,total_data_rech_amt_8,amt_data_6,amt_data_7,amt_data_8,total_data_rech_amt_9,amt_data_9,avg_amt_data_6_7
0,7000842753,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,197.385,...,3.58,252.0,252.0,252.0,614.0,504.0,504.0,NaN,NaN,559.0
1,7001865778,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,34.047,...,0.00,NaN,154.0,100.0,NaN,538.0,383.0,NaN,NaN,538.0
2,0,0,0.0,0.0,0.0,0,0,0,0,0.000,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0.0,0.0,0.0,0,0,0,0,0.000,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7000142493,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,261.636,...,0.00,56.0,NaN,NaN,252.0,NaN,NaN,NaN,NaN,252.0


In [18]:
# Checking remaning all col with missing data more than 5%, will perform further EDA on them
col_to_val_before_drop = raw_data1.columns[round(100*(raw_data1.isnull().sum()/len(raw_data1.index)), 2) > 5.00 ]
col_to_val_before_drop

Index(['onnet_mou_9', 'offnet_mou_9', 'roam_ic_mou_9', 'roam_og_mou_9',
       'loc_og_t2t_mou_9', 'loc_og_t2m_mou_9', 'loc_og_t2f_mou_9',
       'loc_og_t2c_mou_9', 'loc_og_mou_9', 'std_og_t2t_mou_9',
       'std_og_t2m_mou_9', 'std_og_t2f_mou_9', 'std_og_t2c_mou_9',
       'std_og_mou_9', 'isd_og_mou_9', 'spl_og_mou_9', 'og_others_9',
       'loc_ic_t2t_mou_9', 'loc_ic_t2m_mou_9', 'loc_ic_t2f_mou_9',
       'loc_ic_mou_9', 'std_ic_t2t_mou_9', 'std_ic_t2m_mou_9',
       'std_ic_t2f_mou_9', 'std_ic_t2o_mou_9', 'std_ic_mou_9', 'spl_ic_mou_9',
       'isd_ic_mou_9', 'ic_others_9', 'total_rech_data_6', 'total_rech_data_7',
       'total_rech_data_8', 'total_rech_data_9', 'max_rech_data_6',
       'max_rech_data_7', 'max_rech_data_8', 'max_rech_data_9',
       'count_rech_2g_6', 'count_rech_2g_7', 'count_rech_2g_8',
       'count_rech_2g_9', 'count_rech_3g_6', 'count_rech_3g_7',
       'count_rech_3g_8', 'count_rech_3g_9', 'av_rech_amt_data_6',
       'av_rech_amt_data_7', 'av_rech_amt_dat

### Data Preparation

In [19]:
# Creating new attribute to check avg revenue change from good months (6 and 7) to 8th month for both data and voice
raw_data1['arpu_check'] = (((raw_data1.arpu_6 + raw_data1.arpu_7)/2) - raw_data1.arpu_8)
raw_data['arpu_3g_check'] = (((raw_data1.arpu_3g_6 + raw_data1.arpu_3g_7)/2) - raw_data1.arpu_3g_8)
raw_data['arpu_2g_check'] = (((raw_data1.arpu_2g_6 + raw_data1.arpu_2g_7)/2) - raw_data1.arpu_2g_8)

In [20]:
#raw_data1['total_amt_rech_6_7'] = raw_data1[['total_rech_amt_6', 'total_rech_amt_7']].sum(axis=1)

In [21]:
#raw_data1['avg_rec_amount_7'] =  raw_data1.total_rech_amt_7 / raw_data1.total_rech_num_7
#raw_data1['avg_rec_amount_6'] =  (raw_data1.total_rech_amt_6) / (raw_data1.total_rech_num_6)

#raw_data1['avg_rec_amount'] =  (raw_data1.avg_rec_amount_6 + raw_data1.avg_rec_amount_7 ) / 2

#raw_data1['avg_rec_amount1'] =  ((raw_data1.total_rech_amt_6)+ (raw_data1.total_rech_amt_7)) / ((raw_data1.total_rech_num_6) + ((raw_data1.total_rech_num_7)))

#raw_data1['avg_rec_amount2'] =  ((raw_data1.total_rech_amt_6)+ (raw_data1.total_rech_amt_7)) / 2

In [148]:
raw_data1['total_data_rech_amt_6'] = (raw_data1['total_rech_data_6']) * (raw_data1['av_rech_amt_data_6'])
raw_data1['total_data_rech_amt_7'] = (raw_data1['total_rech_data_7']) * (raw_data1['av_rech_amt_data_7'])
raw_data1['total_data_rech_amt_8'] = (raw_data1['total_rech_data_8']) * (raw_data1['av_rech_amt_data_8'])
raw_data1['total_data_rech_amt_9'] = (raw_data1['total_rech_data_9']) * (raw_data1['av_rech_amt_data_9'])
raw_data1['amt_data_6'] = raw_data1['total_rech_amt_6'] + raw_data1['total_data_rech_amt_6']
raw_data1['amt_data_7'] = raw_data1['total_rech_amt_7'] + raw_data1['total_data_rech_amt_7']
raw_data1['amt_data_8'] = raw_data1['total_rech_amt_8'] + raw_data1['total_data_rech_amt_8']
raw_data1['amt_data_9'] = raw_data1['total_rech_amt_9'] + raw_data1['total_data_rech_amt_9']
#where_are_NaNs = np.isnan(raw_data11['avg'])
#raw_data11[where_are_NaNs] = 0


In [150]:
#Avg amount spent in 6th and 7th month
#raw_data11['avg_amount'] = raw_data11[['total_rech_amt_6', 'total_rech_amt_7']].mean(axis=1)
raw_data1['avg_amt_data_6_7'] = raw_data1[['amt_data_6', 'amt_data_7']].mean(axis=1)
where_are_NaNs = np.isnan(raw_data1['avg_amt_data_6_7'])
raw_data1[where_are_NaNs] = 0
#raw_data1['avg_amt_data_6_7'] = (raw_data1['amt_data_6'] + raw_data1['amt_data_7'])/2
list4 = raw_data1['avg_amt_data_6_7'].tolist()
cutoff_rec_amount3 = np.percentile(list4,70)
cutoff_rec_amount3

249.6999999999971

In [153]:
raw_data1.avg_amt_data_6_7.isnull().sum()


0

In [23]:
#list1 = raw_data1['avg_rec_amount'].tolist()

#list2 = raw_data1['avg_rec_amount1'].tolist()

#list3 = raw_data1['avg_rec_amount2'].tolist()


#where_are_NaNs = np.isnan(raw_data1['avg'])
#raw_data1[where_are_NaNs] = 0

#cutoff_rec_amount = np.percentile(list1,70)
#cutoff_rec_amount

#cutoff_rec_amount1 = np.percentile(list2,70)
#cutoff_rec_amount1

#cutoff_rec_amount2 = np.percentile(list3,70)
#cutoff_rec_amount2

In [24]:
#m = raw_data1['avg'].quantile(0.70)
#m

In [25]:
#df1 = raw_data1[['max_rech_amt_6',
# 'max_rech_amt_7','max_rech_amt_8','max_rech_amt_9']]

In [26]:
#df4 = raw_data1[['total_rech_amt_6', 'total_rech_amt_7']]

In [27]:
#df4.head()

In [28]:
#raw_data1['total_rech_amt_8'] = df4.max(axis = 1)

#raw_data1['max_rec_amount1'] = df4.max(axis = 1)
#raw_data1['max_rec_amount1'] = raw_data1['max_rec_amt_']

In [29]:
#raw_data1.max_rec_amount1.values.tolist()

In [30]:
df2 = raw_data1[['total_rech_num_6',
 'total_rech_num_7',
 'total_rech_num_8',
 'total_rech_num_9',
 'total_rech_amt_6',
 'total_rech_amt_7',
 'total_rech_amt_8',
 'total_rech_amt_9',
 'avg_amount','arpu_check','total_amt_rech_6_7']]

In [31]:
round(100*(df2.isnull().sum()/len(df2.index)), 2)

total_rech_num_6      0.0
total_rech_num_7      0.0
total_rech_num_8      0.0
total_rech_num_9      0.0
total_rech_amt_6      0.0
total_rech_amt_7      0.0
total_rech_amt_8      0.0
total_rech_amt_9      0.0
avg_amount            0.0
arpu_check            0.0
total_amt_rech_6_7    0.0
dtype: float64

In [185]:
#df_final = raw_data1[(raw_data1['total_rech_amt_9'] >= cutoff_rec_amount3)|(raw_data1['total_rech_amt_8'] >= cutoff_rec_amount3) |(raw_data1['total_rech_amt_6'] >= cutoff_rec_amount3) | (raw_data1['total_rech_amt_7'] >= cutoff_rec_amount3)]

In [100]:
#df_final = raw_data1[(raw_data1['total_rech_amt_8'] >= cutoff_rec_amount3) | (raw_data1['total_rech_amt_9'] >= cutoff_rec_amount3)]

In [176]:
#df_final = raw_data1[(raw_data1['total_rech_amt_8'] >= cutoff_rec_amount3)]

In [103]:
#df_final = raw_data[(raw_data['total_rech_amt_8'] >= cutoff_rec_amount3)]

In [221]:
df_final = raw_data1[(raw_data1['amt_data_7'] >= cutoff_rec_amount3) | (raw_data1['amt_data_6'] >= cutoff_rec_amount3)]

In [228]:
df_final = raw_data1[(raw_data1['amt_data_7'] >= cutoff_rec_amount3) | (raw_data1['amt_data_6'] >= cutoff_rec_amount3)|(raw_data1['amt_data_8'] >= cutoff_rec_amount3)]

In [229]:
df_final.head()

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,arpu_6,...,sep_vbc_3g,total_data_rech_amt_6,total_data_rech_amt_7,total_data_rech_amt_8,amt_data_6,amt_data_7,amt_data_8,total_data_rech_amt_9,amt_data_9,avg_amt_data_6_7
0,7000842753,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,197.385,...,3.58,252.0,252.0,252.0,614.0,504.0,504.0,NaN,NaN,559.0
1,7001865778,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,34.047,...,0.00,NaN,154.0,100.0,NaN,538.0,383.0,NaN,NaN,538.0
4,7000142493,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,261.636,...,0.00,56.0,NaN,NaN,252.0,NaN,NaN,NaN,NaN,252.0
8,7001524846,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,378.721,...,0.00,NaN,354.0,207.0,NaN,955.0,327.0,NaN,NaN,955.0
9,7001864400,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,119.518,...,0.00,NaN,154.0,NaN,NaN,349.0,NaN,NaN,NaN,349.0


In [230]:
print(len(df_final))

30195


In [231]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30195 entries, 0 to 99998
Columns: 222 entries, mobile_number to avg_amt_data_6_7
dtypes: float64(179), int64(35), object(8)
memory usage: 51.4+ MB


In [232]:
round(100*(df_final.isnull().sum()/len(df_final.index)), 2)

mobile_number             0.00
circle_id                 0.00
loc_og_t2o_mou            1.34
std_og_t2o_mou            1.34
loc_ic_t2o_mou            1.34
last_date_of_month_6      0.00
last_date_of_month_7      0.16
last_date_of_month_8      0.71
last_date_of_month_9      1.38
arpu_6                    0.00
arpu_7                    0.00
arpu_8                    0.00
arpu_9                    0.00
onnet_mou_6               3.45
onnet_mou_7               3.44
onnet_mou_8               5.67
onnet_mou_9               8.04
offnet_mou_6              3.45
offnet_mou_7              3.44
offnet_mou_8              5.67
offnet_mou_9              8.04
roam_ic_mou_6             3.45
roam_ic_mou_7             3.44
roam_ic_mou_8             5.67
roam_ic_mou_9             8.04
roam_og_mou_6             3.45
roam_og_mou_7             3.44
roam_og_mou_8             5.67
roam_og_mou_9             8.04
loc_og_t2t_mou_6          3.45
                         ...  
monthly_2g_6              0.00
monthly_

### Tagging Churn customer as 1 and non churn as 0 based on business condition

In [233]:
df_final['Churn'] = np.where((df_final['total_og_mou_9'] == 0) & (df_final['total_ic_mou_9'] == 0) & (df_final['vol_2g_mb_9'] == 0) & (df_final['vol_3g_mb_9'] == 0),1,0)

C:\Users\NP00544400\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [234]:
df_final['min_usage_9'] = df_final['total_og_mou_9'] + df_final['total_ic_mou_9'] 
df_final['data_usage_9'] = df_final['vol_2g_mb_9']  + df_final['vol_3g_mb_9'] 

C:\Users\NP00544400\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\NP00544400\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [235]:
df_final['Churn1'] = np.where((df_final['min_usage_9'] == 0) & (df_final['data_usage_9'] == 0),1,0)

C:\Users\NP00544400\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [236]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30195 entries, 0 to 99998
Columns: 226 entries, mobile_number to Churn1
dtypes: float64(181), int32(2), int64(35), object(8)
memory usage: 52.1+ MB


In [237]:
df_final.Churn1.isnull().any()

False

In [238]:
df_final.Churn1.value_counts()

0    27681
1     2514
Name: Churn1, dtype: int64

In [239]:
df_final.Churn1.value_counts()

0    27681
1     2514
Name: Churn1, dtype: int64

In [240]:
#Checking % of churn in data
(df_final['Churn1'].value_counts() / df_final['Churn1'].count())*100

0    91.674118
1     8.325882
Name: Churn1, dtype: float64

In [241]:
churn = (sum(df_final['Churn1'])/len(df_final['Churn1'].index))*100
churn

8.32588176850472

In [242]:
#Dropping all attributes from 9th month

Churn_df = df_final[df_final.columns[~df_final.columns.str.endswith('_9')]]

In [243]:
Churn_df.head()

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,...,sep_vbc_3g,total_data_rech_amt_6,total_data_rech_amt_7,total_data_rech_amt_8,amt_data_6,amt_data_7,amt_data_8,avg_amt_data_6_7,Churn,Churn1
0,7000842753,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,197.385,214.816,...,3.58,252.0,252.0,252.0,614.0,504.0,504.0,559.0,1,1
1,7001865778,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,34.047,355.074,...,0.00,NaN,154.0,100.0,NaN,538.0,383.0,538.0,0,0
4,7000142493,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,261.636,309.876,...,0.00,56.0,NaN,NaN,252.0,NaN,NaN,252.0,0,0
8,7001524846,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,378.721,492.223,...,0.00,NaN,354.0,207.0,NaN,955.0,327.0,955.0,0,0
9,7001864400,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,119.518,247.435,...,0.00,NaN,154.0,NaN,NaN,349.0,NaN,349.0,0,0


In [267]:
col_null = Churn_df.columns[round(100*(Churn_df.isnull().sum()/len(Churn_df.index)), 2) > 3 ]
col_null

Index(['onnet_mou_6', 'onnet_mou_7', 'offnet_mou_6', 'offnet_mou_7',
       'roam_ic_mou_6', 'roam_ic_mou_7', 'roam_og_mou_6', 'roam_og_mou_7',
       'loc_og_t2t_mou_6', 'loc_og_t2t_mou_7', 'loc_og_t2m_mou_6',
       'loc_og_t2m_mou_7', 'loc_og_t2f_mou_6', 'loc_og_t2f_mou_7',
       'loc_og_t2c_mou_6', 'loc_og_t2c_mou_7', 'std_og_t2t_mou_6',
       'std_og_t2t_mou_7', 'std_og_t2m_mou_6', 'std_og_t2m_mou_7',
       'std_og_t2f_mou_6', 'std_og_t2f_mou_7', 'std_og_t2c_mou_6',
       'std_og_t2c_mou_7', 'loc_ic_t2t_mou_6', 'loc_ic_t2t_mou_7',
       'loc_ic_t2m_mou_6', 'loc_ic_t2m_mou_7', 'loc_ic_t2f_mou_6',
       'loc_ic_t2f_mou_7', 'loc_ic_mou_6', 'loc_ic_mou_7', 'std_ic_t2t_mou_6',
       'std_ic_t2t_mou_7', 'std_ic_t2m_mou_6', 'std_ic_t2m_mou_7',
       'std_ic_t2f_mou_6', 'std_ic_t2f_mou_7', 'std_ic_t2o_mou_6',
       'std_ic_t2o_mou_7', 'std_ic_mou_6', 'std_ic_mou_7', 'spl_ic_mou_6',
       'spl_ic_mou_7', 'isd_ic_mou_6', 'isd_ic_mou_7', 'ic_others_6',
       'ic_others_7'],
      

In [256]:
Churn_df.drop(['onnet_mou_8', 'offnet_mou_8', 'roam_ic_mou_8', 'roam_og_mou_8',
       'loc_og_t2t_mou_8', 'loc_og_t2m_mou_8', 'loc_og_t2f_mou_8',
       'loc_og_t2c_mou_8', 'loc_og_mou_8', 'std_og_t2t_mou_8',
       'std_og_t2m_mou_8', 'std_og_t2f_mou_8', 'std_og_t2c_mou_8',
       'std_og_mou_8', 'isd_og_mou_8', 'spl_og_mou_8', 'og_others_8',
       'loc_ic_t2t_mou_8', 'loc_ic_t2m_mou_8', 'loc_ic_t2f_mou_8',
       'loc_ic_mou_8', 'std_ic_t2t_mou_8', 'std_ic_t2m_mou_8',
       'std_ic_t2f_mou_8', 'std_ic_t2o_mou_8', 'std_ic_mou_8', 'spl_ic_mou_8',
       'isd_ic_mou_8', 'ic_others_8', 'total_rech_data_6', 'total_rech_data_7',
       'total_rech_data_8', 'max_rech_data_6', 'max_rech_data_7',
       'max_rech_data_8', 'count_rech_2g_6', 'count_rech_2g_7',
       'count_rech_2g_8', 'count_rech_3g_6', 'count_rech_3g_7',
       'count_rech_3g_8', 'av_rech_amt_data_6', 'av_rech_amt_data_7',
       'av_rech_amt_data_8', 'arpu_3g_6', 'arpu_3g_7', 'arpu_3g_8',
       'arpu_2g_6', 'arpu_2g_7', 'arpu_2g_8', 'total_data_rech_amt_6',
       'total_data_rech_amt_7', 'total_data_rech_amt_8', 'amt_data_6',
       'amt_data_7', 'amt_data_8'], axis = 1 , inplace = True)

C:\Users\NP00544400\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [265]:

Churn_df = Churn_df[Churn_df.columns[~Churn_df.columns.str.contains('loc_og_mou_')]]
Churn_df = Churn_df[Churn_df.columns[~Churn_df.columns.str.contains('std_og_mou_')]]
Churn_df = Churn_df[Churn_df.columns[~Churn_df.columns.str.contains('isd_og_mou_')]]
Churn_df = Churn_df[Churn_df.columns[~Churn_df.columns.str.contains('spl_og_mou_')]]
Churn_df = Churn_df[Churn_df.columns[~Churn_df.columns.str.contains('og_others_')]]
Churn_df = Churn_df.loc[:, (Churn_df != 0).any(axis=0)]
Churn_df.head()

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,...,sachet_3g_7,sachet_3g_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g,avg_amt_data_6_7,Churn,Churn1
0,7000842753,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,197.385,214.816,...,0,0,968,30.40,0.00,101.20,3.58,559.0,1,1
1,7001865778,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,34.047,355.074,...,0,0,1006,0.00,0.00,0.00,0.00,538.0,0,0
4,7000142493,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,261.636,309.876,...,0,0,1526,0.00,0.00,0.00,0.00,252.0,0,0
8,7001524846,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,378.721,492.223,...,0,0,315,21.03,910.65,122.16,0.00,955.0,0,0
9,7001864400,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,119.518,247.435,...,0,0,902,0.00,0.00,0.00,0.00,349.0,0,0


In [268]:
df1 =  Churn_df.select_dtypes(include=['float64', 'int64','int','int32'])
#df1.drop('Churn',axis=1, inplace=True)
df1.head()

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,...,sachet_3g_7,sachet_3g_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g,avg_amt_data_6_7,Churn,Churn1
0,7000842753,109,0.0,0.0,0.0,197.385,214.816,213.803,NaN,NaN,...,0,0,968,30.40,0.00,101.20,3.58,559.0,1,1
1,7001865778,109,0.0,0.0,0.0,34.047,355.074,268.321,24.11,78.68,...,0,0,1006,0.00,0.00,0.00,0.00,538.0,0,0
4,7000142493,109,0.0,0.0,0.0,261.636,309.876,238.174,50.31,149.44,...,0,0,1526,0.00,0.00,0.00,0.00,252.0,0,0
8,7001524846,109,0.0,0.0,0.0,378.721,492.223,137.362,413.69,351.03,...,0,0,315,21.03,910.65,122.16,0.00,955.0,0,0
9,7001864400,109,0.0,0.0,0.0,119.518,247.435,170.231,33.89,30.11,...,0,0,902,0.00,0.00,0.00,0.00,349.0,0,0


In [271]:
df1 = df1.fillna(0)
df1 = df1.loc[:, (df1 != 0).any(axis=0)]
df1.drop(['mobile_number','circle_id'], axis=1, inplace=True)
df1.head()

ValueError: labels ['mobile_number' 'circle_id'] not contained in axis

In [272]:
df1 = df1.fillna(0)